In [1]:
import numpy as np
import pandas as pd

# Load Dataset

In [2]:
trainPath = './data/adult.train.10k.discrete'
testPath = './data/adult.test.10k.discrete'

In [3]:
train_df = pd.read_csv(trainPath, header=None)
test_df = pd.read_csv(testPath, header=None)

In [4]:
train_df

,0,1,2,3,4,5,6,7,8
0,<=50K,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,<=50K,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,<=50K,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,<=50K,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,<=50K,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...,...
9995,>50K,Federal-gov,Some-college,Married-civ-spouse,Adm-clerical,Husband,White,Male,United-States
9996,<=50K,Private,Some-college,Married-civ-spouse,Other-service,Wife,Black,Female,United-States
9997,<=50K,Private,Some-college,Married-civ-spouse,Sales,Husband,White,Male,United-States
9998,<=50K,Private,HS-grad,Married-civ-spouse,Adm-clerical,Husband,White,Male,United-States


In [5]:
train_df.describe()

,0,1,2,3,4,5,6,7,8
count,10000,10000,10000,10000,10000,10000,10000,10000,10000
unique,2,7,16,7,14,6,5,2,40
top,<=50K,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,7550,7379,3279,4651,1327,4104,8602,6784,9091


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       10000 non-null  object
 1   1       10000 non-null  object
 2   2       10000 non-null  object
 3   3       10000 non-null  object
 4   4       10000 non-null  object
 5   5       10000 non-null  object
 6   6       10000 non-null  object
 7   7       10000 non-null  object
 8   8       10000 non-null  object
dtypes: object(9)
memory usage: 703.2+ KB


# Decision Tree

In [7]:
class Node:
    def __init__(self, data, data_label, label, features):
        self.data_label = data_label
        self.data = data
        self.label = label
        self.features = features
        self.children = {}
        self.split_feature = None

def entropy(label):
    _, counts = np.unique(label, return_counts=True)
    probabilities = counts / len(label)
    return -np.sum(probabilities * np.log2(probabilities + 1e-10)) # for zero probability

def information_gain(data, label, feature):
    unique_values = np.unique(data[feature])
    total_entropy = entropy(label)

    weighted_entropy = 0
    for value in unique_values:
        subset_indices = data.index[data[feature] == value]
        subset_label = label.loc[subset_indices]
        weighted_entropy += len(subset_label) / len(label) * entropy(subset_label)

    return total_entropy - weighted_entropy

def get_best_feature(data, label, features):
    information_gains = [information_gain(data, label, feature) for feature in features]
    best_feature_index = np.argmax(information_gains)
    return features[best_feature_index]

def id3(data, label, features):
    # If all examples have the same class, create a leaf node
    if len(np.unique(label)) == 1:
        return Node(data, label, label.iloc[0], None)

    # If there are no features left, create a leaf node with the majority class
    if len(features) == 0:
        majority_class = label.value_counts().idxmax()
        return Node(data, label, majority_class, None)

    # Select the best feature to split on
    best_feature = get_best_feature(data, label, features)

    # Create a node for the best feature
    node = Node(data, label, None, features)
    node.split_feature = best_feature

    # Recursively create child nodes for each value of the best feature
    unique_values = np.unique(data[best_feature])
    for value in unique_values:
        subset_indices = data.index[data[best_feature] == value]
        subset_data = data.loc[subset_indices]
        subset_label = label.loc[subset_indices]
        subset_features = [f for f in features if f != best_feature]

        child_node = id3(subset_data, subset_label, subset_features)
        node.children[value] = child_node

    return node

def predict(testData, node):

    predictions = []

    for id, test_data in testData.iterrows():

        current_node = node

        while current_node.children:
            feature_value = test_data[current_node.split_feature]
            if feature_value in current_node.children:
                current_node = current_node.children[feature_value]
            else:
                break
        
        if current_node.label is not None:
            predictions.append(current_node.label)
        else:
            # If the class is not determined, use the majority class of the original data
            majority_class = current_node.data_label.mode().iloc[0]
            predictions.append(majority_class)

    return predictions

In [8]:
labels_column = 0
features_columns = list(range(1, len(train_df.columns)))

train_data = train_df.iloc[:, features_columns]
train_labels = train_df.iloc[:, labels_column]

test_data = test_df.iloc[:, features_columns]
test_labels = test_df.iloc[:, labels_column]

root_node = id3(train_data, train_labels, features_columns)

# Results

In [9]:
trainPrediction = predict(train_data, root_node)
trainAccuracy = sum([element1 == element2 for element1, element2 in zip(train_labels.to_list(), trainPrediction)])/len(trainPrediction) * 100

print(f'Train data Accuracy is : {trainAccuracy}')

Train data Accuracy is : 87.53999999999999


In [10]:
testPrediction = predict(test_data, root_node)
testAccuracy = sum([element1 == element2 for element1, element2 in zip(test_labels.to_list(), testPrediction)])/len(testPrediction) * 100

print(f'Test data Accuracy is : {testAccuracy}')

Test data Accuracy is : 81.23
